In [ ]:
%matplotlib inline
import pylab
import matplotlib.pyplot as plt
import cv2
import numpy as np
import imageio
from itertools import izip

filename = '../CarND-LaneLines-P1/test_images/solidYellowCurve2.jpg'
def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)


def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, [vertices], ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


bgr = []
lines = []
minLineLength = 10
maxLineGap = 2
kernel = np.ones((5,5))

print "read original image"

img = plt.imread(filename)
print img.shape
plt.figure(figsize=(20,20))
plt.imshow(img)
plt.show()

print "bgr version"
colorbase = img.copy()
colorbase2 = img.copy()
bgr = img[...,::-1]
plt.figure(figsize=(20,20))
plt.imshow(bgr)
plt.show()

print "gray scale version"
gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
plt.figure(figsize=(20,20))
plt.imshow(gray, cmap="gray")
plt.show()

print "gaussian blur"
blurred=cv2.GaussianBlur(gray,(7,7),0)
plt.figure(figsize=(20,20))
plt.imshow(blurred, cmap="gray")
plt.show()

print "after canny"
canny=cv2.Canny(blurred, 50,150,3)
plt.figure(figsize=(20,20))
plt.imshow(canny,cmap="gray")
plt.show()

print "masked"
#vertices =np.array([[150,540],[460,320],[500,320],[960,540]], np.int32)
vertices =np.array([[150,540],[460,310],[500,310],[960,540]], np.int32)
masked = region_of_interest(canny, vertices)
#cv2.fillPoly(canny, [vertices], 255)

plt.figure(figsize=(20,20))
plt.imshow(masked,cmap="gray")
plt.show()

slope1 = []
slope2 = []
lines2 = []
lines3 = []

fit_x = []
fit_y = []
real_slope = 0
real_intercept = 0

fit_x2 = []
fit_y2 = []
real_slope2 = 0
real_intercept2 = 0

lines = cv2.HoughLinesP(masked, 1, np.pi/180, 25, minLineLength, maxLineGap)

#print "all hough lines", lines
for line in lines:
    for i,[x1,y1,x2,y2] in enumerate(line):
        temp_slope=((y2-y1)/float(x2-x1))
        
        if temp_slope >= 0.3:
            lines2.append([x1,y1,x2,y2])
            slope1.append(temp_slope)
        elif temp_slope <= -0.6:
            lines3.append([x1,y1,x2,y2])
            slope2.append(temp_slope)

#print  sorted(np.array(slope1))
#print  sorted(np.array(slope2))

print "left mean", np.array(slope1).mean()
print "left std dev", np.array(slope1).std()

print "right mean", np.array(slope2).mean()
print "right std dev", np.array(slope2).std()


lines2 = np.expand_dims(np.array(lines2), axis=0)
lines3 = np.expand_dims(np.array(lines3), axis=0)

# sorting points
lines2_srtd = sorted(lines2[0], key=lambda k:k[3])
lines3_srtd = sorted(lines3[0], key=lambda k:k[3])

lines2_srtd = np.expand_dims(np.array(lines2_srtd), axis=0)
lines3_srtd = np.expand_dims(np.array(lines3_srtd), axis=0)

print "hough lines"                   
draw_lines(colorbase, lines2, color=[255, 0, 0], thickness=2)
draw_lines(colorbase, lines3, color=[0, 255, 0], thickness=2)
plt.figure(figsize=(20,20))
plt.imshow(colorbase,cmap="gray")
plt.show()

print "searching for the line of best fit"

print "left line"
#for line in lines2_srtd:
#    print line

for line in lines2_srtd:
    for x1,y1,x2,y2 in line:
        #print "oryg", x1,y1,x2,y2
        av_x = (x2+x1)/2
        #print "avg x",av_x
        av_y = (y2+y1)/2
        #print "avg y",av_y
        fit_x.append(av_x)
        fit_y.append(av_y)


real_slope, real_intercept = np.polyfit(fit_x,fit_y,1)
print "m",real_slope
print "b", real_intercept

# fix left line so it gets to the bottom of the image
fit_x.append(960)

real_y = [int(real_slope * x + real_intercept) for x in fit_x]
#print real_y

x0 = fit_x[0]
y0 = real_y[0]

for x,y in izip(fit_x, real_y):
    #print x0,y0,x,y
    cv2.line(colorbase2, (x0, y0), (x, y), (0,0,255), 2)
    x0=x
    y0=y


print "right line"
#for line in lines3_srtd:
#    print line
    
    
for line in lines3_srtd:
    for x1,y1,x2,y2 in line:
        #print "oryg", x1,y1,x2,y2
        av_x = (x2+x1)/2
        #print "avg x",av_x
        av_y = (y2+y1)/2
        #print "avg y",av_y
        fit_x2.append(av_x)
        fit_y2.append(av_y)


real_slope2, real_intercept2 = np.polyfit(fit_x2,fit_y2,1)
print "m",real_slope2
print "b", real_intercept2

# fix left line so it gets to the bottom of the image
fit_x2.append(0)

real_y2 = [int(real_slope2 * x + real_intercept2) for x in fit_x2]
#print real_y2

x0 = fit_x2[0]
y0 = real_y2[0]

for x,y in izip(fit_x2, real_y2):
    #print x0,y0,x,y
    cv2.line(colorbase2, (x0, y0), (x, y), (0,255,255), 2)
    x0=x
    y0=y    


plt.figure(figsize=(20,20))
plt.imshow(colorbase2)
plt.show()

